In [3]:
import sys
from pathlib import Path
project_root = Path.cwd().resolve().parent  # or .parent.parent 依据你的目录
sys.path.insert(0, str(project_root))

from holopart.pipelines.pipeline_holopart import HoloPartPipeline
from holopart.inference_utils import hierarchical_extract_geometry, flash_extract_geometry
from huggingface_hub import snapshot_download
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
dtype = torch.float16


In [8]:
holopart_weights_dir = "pretrained_weights/HoloPart"
snapshot_download(repo_id="VAST-AI/HoloPart", local_dir=holopart_weights_dir)

Fetching 10 files:  10%|█         | 1/10 [00:01<00:09,  1.07s/it]'(MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /VAST-AI/HoloPart/resolve/29cdb1417121842c4c4d59b9d283cc734f8c3aec/transformer/diffusion_pytorch_model.safetensors (Caused by SSLError(SSLEOFError(8, '[SSL: UNEXPECTED_EOF_WHILE_READING] EOF occurred in violation of protocol (_ssl.c:1017)')))"), '(Request ID: 17b67810-b446-41a2-85cf-4e69d8bb0afb)')' thrown while requesting HEAD https://huggingface.co/VAST-AI/HoloPart/resolve/29cdb1417121842c4c4d59b9d283cc734f8c3aec/transformer/diffusion_pytorch_model.safetensors
Retrying in 1s [Retry 1/5].
Fetching 10 files: 100%|██████████| 10/10 [00:28<00:00,  2.84s/it]


'/mnt/d/wsl/seulxia/HoloPart/pretrained_weights/HoloPart'

In [13]:
HoloPartPipeline = HoloPartPipeline.from_pretrained(holopart_weights_dir).to(device, dtype)

Loading pipeline components...: 100%|██████████| 4/4 [00:38<00:00,  9.72s/it]


In [14]:
print(HoloPartPipeline)

HoloPartPipeline {
  "_class_name": "HoloPartPipeline",
  "_diffusers_version": "0.35.2",
  "_name_or_path": "pretrained_weights/HoloPart",
  "part_encoder": [
    "holopart.models.part_encoders",
    "PartEncoder"
  ],
  "scheduler": [
    "holopart.schedulers.scheduling_rectified_flow",
    "RectifiedFlowScheduler"
  ],
  "transformer": [
    "holopart.models.transformers.triposg_transformer",
    "TripoSGDiTModel"
  ],
  "vae": [
    "holopart.models.autoencoders.autoencoder_kl_triposg",
    "TripoSGVAEModel"
  ]
}



In [25]:
HoloPartPipeline.part_encoder.name_parameters()

AttributeError: 'PartEncoder' object has no attribute 'name_parameters'

In [19]:
# 方法1: 查看所有参数的总数量和每个组件的参数数量
def count_parameters(model):
    """统计模型参数数量"""
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    return total_params, trainable_params

print("=" * 80)
print("各组件参数统计:")
print("=" * 80)

# 统计各个组件
components = {
    'VAE': HoloPartPipeline.vae,
    'Transformer': HoloPartPipeline.transformer,
    'Part Encoder': HoloPartPipeline.part_encoder,
}

total_all = 0
for name, component in components.items():
    if component is not None:
        total, trainable = count_parameters(component)
        total_all += total
        print(f"{name}:")
        print(f"  总参数: {total:,} ({total/1e6:.2f}M)")
        print(f"  可训练参数: {trainable:,} ({trainable/1e6:.2f}M)")
        print()

print("=" * 80)
print(f"Pipeline总参数: {total_all:,} ({total_all/1e6:.2f}M)")
print("=" * 80)


各组件参数统计:
VAE:
  总参数: 242,662,017 (242.66M)
  可训练参数: 242,662,017 (242.66M)

Transformer:
  总参数: 1,615,851,328 (1615.85M)
  可训练参数: 1,615,851,328 (1615.85M)

Part Encoder:
  总参数: 56,804,352 (56.80M)
  可训练参数: 56,804,352 (56.80M)

Pipeline总参数: 1,915,317,697 (1915.32M)


In [24]:
# 方法2: 查看各组件的配置
print("=" * 80)
print("各组件的配置信息:")
print("=" * 80)

# VAE配置
if hasattr(HoloPartPipeline.vae, 'config'):
    print("\nVAE配置:")
    for key, value in HoloPartPipeline.vae.config.items():
        print(f"  {key}: {value}")

# Transformer配置
if hasattr(HoloPartPipeline.transformer, 'config'):
    print("\nTransformer配置:")
    for key, value in HoloPartPipeline.transformer.config.items():
        print(f"  {key}: {value}")

# Part Encoder配置
if hasattr(HoloPartPipeline.part_encoder, 'config'):
    print("\nPart Encoder配置:")
    for key, value in HoloPartPipeline.part_encoder.config.items():
        print(f"  {key}: {value}")

# Scheduler配置
if hasattr(HoloPartPipeline.scheduler, 'config'):
    print("\nScheduler配置:")
    for key, value in HoloPartPipeline.scheduler.config.items():
        print(f"  {key}: {value}")


各组件的配置信息:

VAE配置:
  in_channels: 3
  latent_channels: 64
  num_attention_heads: 8
  width_encoder: 512
  width_decoder: 1024
  num_layers_encoder: 8
  num_layers_decoder: 16
  embedding_type: frequency
  embed_frequency: 8
  embed_include_pi: False
  _class_name: TripoSGVAEModel
  _diffusers_version: 0.30.3
  _name_or_path: pretrained_weights/HoloPart/vae

Transformer配置:
  num_attention_heads: 16
  width: 2048
  in_channels: 64
  num_layers: 21
  cross_attention_dim: 512
  cross_attention_2_dim: 512
  use_cross_attention: True
  use_cross_attention_2: True
  _class_name: TripoSGDiTModel
  _diffusers_version: 0.30.3
  _name_or_path: pretrained_weights/HoloPart/transformer

Part Encoder配置:
  position_channels: 3
  part_feature_channels: 3
  whole_feature_channels: 4
  dim: 512
  num_attention_heads: 8
  num_layers: 8
  num_tokens: 2048
  embedding_type: frequency
  embed_frequency: 8
  embed_include_pi: False
  part_local: True
  init_weights: None
  noise_level: None
  _class_name: Part

In [26]:
# 查看 pipeline 的所有属性
print([attr for attr in dir(HoloPartPipeline) if not attr.startswith('_')])

# 检查是否有 model 属性
if hasattr(HoloPartPipeline, 'model'):
    print("Pipeline has model attribute")
    model = HoloPartPipeline.model
    for name, param in model.named_parameters():
        print(name, param.shape)
    
    total_params = sum(p.numel() for p in model.parameters())
    print(f"Total model parameters: {total_params}")

['attention_kwargs', 'components', 'config', 'config_name', 'device', 'disable_attention_slicing', 'disable_vae_slicing', 'disable_vae_tiling', 'disable_xformers_memory_efficient_attention', 'do_classifier_free_guidance', 'download', 'dtype', 'enable_attention_slicing', 'enable_model_cpu_offload', 'enable_sequential_cpu_offload', 'enable_vae_slicing', 'enable_vae_tiling', 'enable_xformers_memory_efficient_attention', 'encode_context_local', 'extract_init_dict', 'from_config', 'from_pipe', 'from_pretrained', 'fuse_qkv_projections', 'get_config_dict', 'guidance_scale', 'has_compatibles', 'hf_device_map', 'ignore_for_config', 'interrupt', 'load_config', 'maybe_free_model_hooks', 'model_cpu_offload_seq', 'name_or_path', 'num_timesteps', 'numpy_to_pil', 'part_encoder', 'prepare_latents', 'progress_bar', 'push_to_hub', 'register_modules', 'register_to_config', 'remove_all_hooks', 'reset_device_map', 'save_config', 'save_pretrained', 'scheduler', 'set_attention_slice', 'set_progress_bar_confi

In [27]:
# 检查 transformer 组件（很可能是主要的模型）
if hasattr(HoloPartPipeline, 'transformer'):
    print("=== Transformer 组件 ===")
    if hasattr(HoloPartPipeline.transformer, 'named_parameters'):
        total_params = 0
        for name, param in HoloPartPipeline.transformer.named_parameters():
            print(f"{name}: {param.shape}")
            total_params += param.numel()
        print(f"Transformer 总参数: {total_params:,}")
    else:
        print("Transformer 没有 named_parameters 方法")

# 检查 VAE 组件
if hasattr(HoloPartPipeline, 'vae'):
    print("\n=== VAE 组件 ===")
    if hasattr(HoloPartPipeline.vae, 'named_parameters'):
        total_params = 0
        for name, param in HoloPartPipeline.vae.named_parameters():
            print(f"{name}: {param.shape}")
            total_params += param.numel()
        print(f"VAE 总参数: {total_params:,}")
    else:
        print("VAE 没有 named_parameters 方法")

# 检查 part_encoder 组件
if hasattr(HoloPartPipeline, 'part_encoder'):
    print("\n=== Part Encoder 组件 ===")
    if hasattr(HoloPartPipeline.part_encoder, 'named_parameters'):
        total_params = 0
        for name, param in HoloPartPipeline.part_encoder.named_parameters():
            print(f"{name}: {param.shape}")
            total_params += param.numel()
        print(f"Part Encoder 总参数: {total_params:,}")
    else:
        print("Part Encoder 没有 named_parameters 方法")

# 计算所有组件的总参数
print("\n=== 总参数统计 ===")
total_all_params = 0
components = []

for attr in ['transformer', 'vae', 'part_encoder']:
    if hasattr(HoloPartPipeline, attr):
        component = getattr(HoloPartPipeline, attr)
        if hasattr(component, 'parameters'):
            component_params = sum(p.numel() for p in component.parameters())
            components.append((attr, component_params))
            total_all_params += component_params

for name, params in components:
    print(f"{name}: {params:,} 参数")

print(f"所有组件总参数: {total_all_params:,}")

=== Transformer 组件 ===
time_proj.linear_1.weight: torch.Size([8192, 2048])
time_proj.linear_1.bias: torch.Size([8192])
time_proj.linear_2.weight: torch.Size([2048, 8192])
time_proj.linear_2.bias: torch.Size([2048])
proj_in.weight: torch.Size([2048, 64])
proj_in.bias: torch.Size([2048])
blocks.0.norm1.weight: torch.Size([2048])
blocks.0.norm1.bias: torch.Size([2048])
blocks.0.attn1.norm_q.weight: torch.Size([128])
blocks.0.attn1.norm_k.weight: torch.Size([128])
blocks.0.attn1.to_q.weight: torch.Size([2048, 2048])
blocks.0.attn1.to_k.weight: torch.Size([2048, 2048])
blocks.0.attn1.to_v.weight: torch.Size([2048, 2048])
blocks.0.attn1.to_out.0.weight: torch.Size([2048, 2048])
blocks.0.attn1.to_out.0.bias: torch.Size([2048])
blocks.0.norm2.weight: torch.Size([2048])
blocks.0.norm2.bias: torch.Size([2048])
blocks.0.attn2.norm_q.weight: torch.Size([128])
blocks.0.attn2.norm_k.weight: torch.Size([128])
blocks.0.attn2.to_q.weight: torch.Size([2048, 2048])
blocks.0.attn2.to_k.weight: torch.Size(